# MRI T1 Processing and Segmentation Documentation

This notebook outlines the workflow for MRI T1 processing and segmentation, specifically tailored for use within Cedalion. Cedalion workflows require either individual or standard head models, built from raw T1 images through a series of segmentation and brain surface extraction steps. This document details the complete processing pipeline for creating standard head models, facilitating the extension of these methods to individual head models.

## Overview of the Processing Workflow

The workflow integrates multiple tools outside Python, including Brainstorm, CAT12, and FreeSurfer, for sequential processing of MRI data. This documentation provides a step-by-step guide to help ensure efficient data preparation, segmentation, and model alignment.

### Step 1: T1 Image Preprocessing with CAT12 in Brainstorm

1. **Brainstorm Overview**:  
   Brainstorm is an open-source application designed for the analysis of brain recordings. It can be used as a standalone version or installed via MATLAB, and is integral to many neuroimaging workflows. It makes it easy to perform CAT12 and Freesurfer analysis.

2. **CAT12 Toolbox**:  
   CAT12 is a computational toolbox within the SPM (Statistical Parametric Mapping) environment, primarily used for structural MRI (sMRI) analysis. CAT12 automates many preprocessing steps required for sMRI, including:
   - **Tissue segmentation** into six types (skull, scalp, cerebrospinal fluid [CSF], gray matter, white matter, and air).
   - **Normalization and modulation** of images.
   - **Surface generation** for brain and head models.

   **Output from CAT12**: For Cedalion, we use CAT12’s outputted 6-type tissue segmentations (skull, scalp, CSF, gray matter, white matter, and air) and the generated head surface.

### Step 2: Detailed Analysis with FreeSurfer

1. **FreeSurfer Overview**:  
   FreeSurfer is a widely-used software package in neuroimaging, specifically for MRI structural analysis. Known for its precise and reproducible methods, FreeSurfer is used to segment brain structures, reconstruct cortical surfaces, and quantify cortical thickness, surface area, and volume.

2. **Processing Outputs**:  
   In this workflow, FreeSurfer provides details including:
   - **Cortical and subcortical segmentation**
   - **Gray and white matter segmentation**
   - **Detailed brain surface model**

   **Key Outputs from FreeSurfer**: Cedalion’s workflow uses FreeSurfer’s gray matter and white matter segmentations and its detailed brain surface model.

### Step 3: Alignment and Optimization in Brainstorm

1. **Data Alignment**:  
   To ensure consistency, outputs from CAT12 and FreeSurfer are loaded into Brainstorm, where tissue segmentation files and surfaces are aligned to MNI coordinates. This step is crucial for integrating data accurately within Cedalion’s models.

2. **Mesh Optimization**:  
   Since FreeSurfer surfaces contain ~300K vertices, we downsample the mesh in Brainstorm to a more manageable 15K vertices, which balances detail and processing efficiency in Cedalion.

### Step 4: Post-Processing of Tissue Segmentation Masks

1. **Post-Processing Workflow**:  
   To finalize tissue segmentation masks, Cedalion applies additional post-processing steps to smoothen boundaries and fill small gaps. These operations ensure cleaner and more continuous tissue delineation.

2. **Preservation of Key Segmentations**:
   - **Gray and White Matter**: FreeSurfer’s gray and white matter segmentations are retained without modification due to their high accuracy. Final checks are applied to prevent overlap between masks and to ensure clarity in tissue delineation.

### Final Outputs

For each T1 image, the workflow yields the following:

- **6 Tissue Segmentation Masks**: skull, scalp, CSF, gray matter, white matter, and air (air mask optional).
- **Head Surface and Brain Surface** 

Note that the brain surface, along with gray and white matter masks, originates from FreeSurfer. All other segmentation masks and the head surface are generated from CAT12.

The flowchart below provides a visual summary of this processing pipeline.

![Alt Text](MRI_processing_diagram.png)

In [31]:
# load dependencies

import pyvista as pv
pv.set_jupyter_backend('client')
#pv.set_jupyter_backend('static')

import os

import cedalion
import cedalion.io
import cedalion.plots
import cedalion.datasets
import cedalion.imagereco.forward_model as fw


import numpy as np
import matplotlib.pyplot as p

import nibabel as nib
from nilearn import plotting


## Colin 27

The Colin27 head is a high-resolution, standardized anatomical brain template based on a series of MRI scans from a single individual, known as “Colin.” The final dataset was created by averaging 27 separate scans of Colin’s brain, resulting in an exceptionally high signal-to-noise ratio and high anatomical detail.


### Limitations of the Colin27 Head
- Single-Subject Template: Because Colin27 is based on a single individual, it may not represent anatomical variations found in a broader population. Templates like MNI152 or the ICBM (International Consortium for Brain Mapping) average, which are based on multiple subjects, may be preferred in population studies.
- Older MRI Technology: The Colin27 scans were acquired in the 1990s using MRI technology of that time, which, while high-quality, may not match the resolution possible with modern imaging techniques.

### Plotting 

### Creating Two-Surface Head Model

#### Using from_surfaces method

This method utilizes the brain surface extracted from Freesurfer analysis and the head surface extracted from CAT12. Note that if either of the surface files are not provided it uses the segmentation files.

In [37]:
# load segmentation data from the colin27 atlas
SEG_DATADIR_cl27, mask_files_cl27, landmarks_file_cl27 = cedalion.datasets.get_colin27_segmentation()

In [32]:

# create forward model class for colin 27 atlas
head_colin27 = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR_cl27,
    mask_files = mask_files_cl27,
    brain_surface_file= os.path.join(SEG_DATADIR_cl27, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR_cl27, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file_cl27,
    brain_face_count=None,
    scalp_face_count=None
)


In [ ]:
# plot Colin headmodel
plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head_colin27.brain, color="w")
cedalion.plots.plot_surface(plt, head_colin27.scalp, opacity=.1)
plt.show()

#### Using from_segmentations method

This method utilizes tissue segmentation files and creates brain and head surfaces from the provided masks.

In [28]:
head_colin27 = fw.TwoSurfaceHeadModel.from_segmentation(
    segmentation_dir=SEG_DATADIR_cl27,
    mask_files = mask_files_cl27,
    landmarks_ras_file=landmarks_file_cl27,
    brain_face_count=None,
    scalp_face_count=None
)

In [ ]:
# plot Colin headmodel
plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head_colin27.brain, color="w")
cedalion.plots.plot_surface(plt, head_colin27.scalp, opacity=.1)
plt.show()

We can also visualize different tisuue segmentation volumes.

In [ ]:
colin27_mask_data = (
    head_colin27.segmentation_masks.sel(segmentation_type='wm') +
    head_colin27.segmentation_masks.sel(segmentation_type='gm') +
    head_colin27.segmentation_masks.sel(segmentation_type='csf') +
    head_colin27.segmentation_masks.sel(segmentation_type='scalp') +
    head_colin27.segmentation_masks.sel(segmentation_type='skull')
)

mask_data = colin27_mask_data.values

# Create a PyVista ImageData object with correct dimensions
grid = pv.ImageData(dimensions=mask_data.shape)
grid.spacing = (1.0, 1.0, 1.0)  # Replace with actual voxel dimensions if available

flattened_mask_data = mask_data.flatten(order='F')

# Choose the specific label to visualize (e.g., Label 2 for 'gm')
label_to_visualize = 5 
binary_mask = np.where(flattened_mask_data == label_to_visualize, 1.0, 0.0)

# Add binary mask to the grid as 'SpecificLabel'
grid.point_data['SpecificLabel'] = binary_mask.astype(float)*100

# Plotting
plotter = pv.Plotter()
# Adding the volume with the appropriate opacity mapping

plotter.add_volume(
    grid, 
    scalars='SpecificLabel', 
    cmap='Greys', 
    clim=[0, 255], 
    opacity=[0.0, 0.3], 
    shade=True, 
    show_scalar_bar=False)

plotter.add_axes()
plotter.show_bounds(grid=True)
plotter.show()


In [ ]:
mri_img = nib.Nifti1Image(colin27_mask_data, affine=head_colin27.t_ras2ijk)

# Plot the Anatomy
plotting.plot_anat(mri_img, display_mode='ortho', title='Tissue Segmentations')
plotting.show()

## ICBM 152

ICBM152 is a template widely used as a standard brain reference in neuroimaging. Unlike the older Colin27, which is based on a single individual's MRI scans, the ICBM152 template is created from the averaged MRI scans of 152 healthy adult brains, which makes it more representative of the general population. 

### Limitations
- Still Population-Based: While ICBM152 2020 is more generalizable than single-subject templates, it still represents an average of a specific sample, which may not perfectly match individual brains, particularly those outside the demographic profile used for the template.

- Not Always Suitable for Pediatric or Clinical Populations: ICBM152 2020 is based on adult brains, so studies involving pediatric, elderly, or clinical populations with specific pathologies may require alternative templates for more accurate representation.

In [39]:
# load segmentation data from the icbm152 atlas
SEG_DATADIR_ic152, mask_files_ic152, landmarks_file_ic152 = cedalion.datasets.get_icbm152_segmentation()

In [44]:
# create forward model class for icbm152 atlas
head_icbm152 = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR_ic152,
    mask_files = mask_files_ic152,
    brain_surface_file= os.path.join(SEG_DATADIR_ic152, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR_ic152, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file_ic152,
    brain_face_count=None,
    scalp_face_count=None
)


In [ ]:
# plot ICBM headmodel
plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head_icbm152.brain, color="w")
cedalion.plots.plot_surface(plt, head_icbm152.scalp, opacity=.1)
plt.show()

In [47]:
# create forward model class for icbm152 atlas
head_icbm152 = fw.TwoSurfaceHeadModel.from_segmentation(
    segmentation_dir=SEG_DATADIR_ic152,
    mask_files = mask_files_ic152,
    landmarks_ras_file=landmarks_file_ic152,
    brain_face_count=None,
    scalp_face_count=None
)


In [ ]:
# plot ICBM headmodel
plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head_icbm152.brain, color="w")
cedalion.plots.plot_surface(plt, head_icbm152.scalp, opacity=.1)
plt.show()

In [ ]:
icbm152_mask_data = (
    head_icbm152.segmentation_masks.sel(segmentation_type='wm') +
    head_icbm152.segmentation_masks.sel(segmentation_type='gm') +
    head_icbm152.segmentation_masks.sel(segmentation_type='csf') +
    head_icbm152.segmentation_masks.sel(segmentation_type='scalp') +
    head_icbm152.segmentation_masks.sel(segmentation_type='skull')
)

mask_data = icbm152_mask_data.values

# Create a PyVista ImageData object with correct dimensions
grid = pv.ImageData(dimensions=mask_data.shape)
grid.spacing = (1.0, 1.0, 1.0)  # Replace with actual voxel dimensions if available

flattened_mask_data = mask_data.flatten(order='F')

# Choose the specific label to visualize (e.g., Label 2 for 'gm')
label_to_visualize = 5 
binary_mask = np.where(flattened_mask_data == label_to_visualize, 1.0, 0.0)

# Add binary mask to the grid as 'SpecificLabel'
grid.point_data['SpecificLabel'] = binary_mask.astype(float)*100

# Plotting
plotter = pv.Plotter()
# Adding the volume with the appropriate opacity mapping

plotter.add_volume(
    grid, 
    scalars='SpecificLabel', 
    cmap='Greys', 
    clim=[0, 255], 
    opacity=[0.0, 0.3], 
    shade=True, 
    show_scalar_bar=False)

plotter.add_axes()
plotter.show_bounds(grid=True)
plotter.show()


In [ ]:
mri_img = nib.Nifti1Image(icbm152_mask_data, affine=head_icbm152.t_ras2ijk)

# Plot the Anatomy
plotting.plot_anat(mri_img, display_mode='ortho', title='Tissue Segmentations')
plotting.show()
